# Observation

## 1. Data Imbalance
training data: 569877(96.5%) v.s. 20663(3.5%)

#### 1.1 Try to balance the data:

##### (Over sampling)

1. simple over sampling 
2. SMOTE

##### (under sampling)
1. simple under sampling
2. Tomek links

#### 1.2 Transfer learning for rare label data

## 2. large class in category features

26 category features class<=10,  23 category features class>10 
(number of class, from 3 to 13553)


##### (if class>10, one hot encoding may suffer from dimension explode
##### choose other encoding methods if class>10 and dis-order features)

有序資料可以直接label encoding

無序資料通常做one hot encoding (ex. xgboost, 例外: LGBM餵label encoding即可)

## 3. plenty of NaN data
Around 180 features get NaN datas over 50%

# To do to improve model performance

### 1. deal with data imbalancing (performance variance in current work, 80-90%)
### 2. change cross-validaiton into walk-through (for time series data)

# Feedback of this project

### important features:
1. card1
2. card2
3. TransactionID
4. TransactionAmt
5. addr1

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.utils import resample

import lightgbm as lgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv


In [2]:
tr_id_path = '/kaggle/input/ieee-fraud-detection/train_identity.csv'
tr_tran_path = '/kaggle/input/ieee-fraud-detection/train_transaction.csv'
te_id_path = '/kaggle/input/ieee-fraud-detection/test_identity.csv'
te_tran_path = '/kaggle/input/ieee-fraud-detection/test_transaction.csv'

In [ ]:
tr_id = pd.read_csv(tr_id_path)
tr_tran = pd.read_csv(tr_tran_path)
te_id = pd.read_csv(te_id_path)
te_tran = pd.read_csv(te_tran_path)

# Data Wrangling & Visualization

In [ ]:
print (tr_id.head)
print (tr_tran.head)

In [ ]:
tr = pd.merge(tr_id, tr_tran, on=['TransactionID'], how='right')
tr.head(10)

# te = pd.merge(te_id, te_tran, on=['TransactionID'], how='right')


In [ ]:
# # Find users

# tr_id_columns = tr_id.columns

# x = tr_id_columns.drop('TransactionID').tolist()
# tr_id_unique = tr.groupby(x, dropna=False).size().reset_index().rename(columns={0:'count'})
# print (tr_id_unique)

# tr_id_unique_fraud = tr[tr['isFraud']==1].groupby(x, dropna=False).size().reset_index().rename(columns={0:'count'})
# sorted_tr_id_unique_fraud = tr_id_unique_fraud.sort_values('count', ascending=False).reset_index(drop=True)
# print (sorted_tr_id_unique_fraud)

# print (sorted_tr_id_unique_fraud.iloc[0])
# print (sorted_tr_id_unique_fraud.iloc[1])
# print (sorted_tr_id_unique_fraud.iloc[2])

clean data

In [ ]:
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan) 
tr = clean_inf_nan(tr)

In [ ]:
tr[tr['isFraud']==0].head(5)

In [ ]:
tr[tr['isFraud']==1].head(5)

In [ ]:
tr_label = tr['isFraud']
tr.drop(columns=["isFraud"], inplace=True)
tr.head(2) 

In [ ]:
del tr_tran,tr_id
del te_tran,te_id

### Data imbalance

In [ ]:
x = tr_label.value_counts().values
print (x)
print (x/sum(x)*100)
print ('Suffer from data imbalancing')
sns.barplot([0,1], x)

In [ ]:
# tr['id_12'].value_counts(dropna=False, normalize=True).values

### category feature processing (large class in category features)

In [ ]:
category_cols=['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9']

len_class_list = []
label_encoder={}
top_important_features=['card1', 'card2', 'addr1', 'P_emaildomain']  ## from results below
for col in category_cols:
    if col in tr.columns:
        le=LabelEncoder()
        le.fit(list(tr[col].astype(str).values) )  # + list(te[col].astype(str).values)
        print (str(col) + ' ' + str(len(le.classes_))+ ' ')
        tr[col] = le.transform(list(tr[col].astype(str).values))
#         te[col] = le.transform(list(te[col].astype(str).values))
        len_class_list.append(len(le.classes_))
        
        if col in top_important_features:
            label_encoder[col] = le

plt.hist(len_class_list, bins=100)
plt.title('Distribution of category features class')

In [ ]:
print (f'There are {len(len_class_list)} category label' )
print ( f'{len([x for x in len_class_list if x>10])} category label, class >10')
print ( f'{len([x for x in len_class_list if x<=10])} category label, class <=10')

In [ ]:
tr['card1']
# tr['card2']

### nan processing

In [ ]:
nan_ratio = (tr.isna().sum())/len(tr)*100
plt.hist(nan_ratio, bins=50)
plt.title('Distribution of features nan_ratio')

In [ ]:
def exclude_features_with_nan_ratio(df, nan_ratio, threshold=50):
    features_column = nan_ratio[nan_ratio.values<threshold].index
    return df[features_column]

def fill_nan_with_mean(df, numerical_columns):
    df[numerical_columns].fillna(df[numerical_columns].mean())
    return df.fillna(df.mean())

numerical_cols = [x for x in tr.columns if x not in category_cols]

tr = exclude_features_with_nan_ratio(tr, nan_ratio)
# tr = fill_nan_with_mean(tr, numerical_cols)

tr.head(10)

In [ ]:
def z_score_normalize(df):
    return (df-df.mean())/df.std()
norm_tr = z_score_normalize(tr)

In [ ]:
# pca=PCA(n_components=2)
# pca.fit(norm_tr)
# pca_result = pca.transform(norm_tr)
# plt.scatter(pca_result[:,0], pca_result[:,1], c=tr_label, s=20, alpha=1, marker='o')
# plt.show()

# pca=PCA(n_components=2)
# pca.fit(norm_tr[numerical_cols])
# pca_result = pca.transform(norm_tr[numerical_cols])
# plt.scatter(pca_result[:,0], pca_result[:,1], c=tr_label, s=20, alpha=1, marker='o')
# plt.show()

# pca=PCA(n_components=2)
# pca.fit(norm_tr[category_cols])
# pca_result = pca.transform(norm_tr[category_cols])
# plt.scatter(pca_result[:,0], pca_result[:,1], c=tr_label, s=20, alpha=1, marker='o')
# plt.show()

In [ ]:
# tsne = TSNE(n_components=2)
# tsne.fit(norm_tr)
# tsne_result = tsne.transform(norm_tr)
# tsne.scatter(tsne_result[:,0], tsne_result[:,1], c=tr_label, s=20, alpha=1, marker='o')
# plt.show()

# Modeling

In [ ]:
# train_x, val_x, train_y, val_y = train_test_split(tr, tr_label, test_size=0.2, random_state=7)

# del tr, tr_label

In [ ]:
params = {'num_leaves': [256],
          'min_child_samples': [79],
          'max_depth': [13],
          'learning_rate': [0.03],
          "subsample_freq": [3],
          "subsample": [0.9],
          "bagging_seed": [11],
          "verbosity": [-1],
          'reg_alpha': [0.3],
          'reg_lambda': [0.3],
          'colsample_bytree': [0.9],
         }

lgbm_classifier = lgb.LGBMClassifier(
          objective= 'binary',
          boosting_type= "gbdt",
          metric= 'auc'
)
gsearch = GridSearchCV(lgbm_classifier, n_jobs=1, param_grid=params, cv = 5, scoring="roc_auc", verbose=10)
gsearch.fit(tr, tr_label)

best_paramters = gsearch.best_estimator_.get_params()

In [ ]:
gsearch.cv_results_

In [ ]:
lgb.plot_importance(gsearch.best_estimator_, max_num_features=20)

# Feedback

In [ ]:
tr_non_fraud = tr[tr_label==0]
tr_fraud = tr[tr_label==1]

In [ ]:
def plot_distribution(feature, tr_non_fraud, tr_fraud, label_encoder=None, up_sampling=True):
    non_fraud = tr_non_fraud[feature]
    fraud = tr_fraud[feature]
    
    if label_encoder:
        non_fraud= label_encoder.inverse_transform(non_fraud)
        fraud = label_encoder.inverse_transform(fraud)
        
        non_fraud = np.array([x for x in non_fraud if x !='nan']).astype(float)
        fraud = np.array([x for x in fraud if x !='nan']).astype(float)
    
    if up_sampling:
        fraud = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(non_fraud), # match number in majority class
                          random_state=1) # reproducible results
    
    
    plt.hist(non_fraud, bins=50)
    plt.title(f'Distribution of non_fraud {feature}')
    plt.hist(fraud, bins=50)
    plt.title(f'Distribution of fraud {feature}')
    plt.show()
    

    ax1=sns.kdeplot(non_fraud,shade=True,label='non_fraud')
    ax2=sns.kdeplot(fraud,shade=True,label='fraud')
    plt.show()

    print (f'Non_fraud, mean:{np.mean(non_fraud)}, std:{np.std(non_fraud)}')
    print (f'fraud, mean:{np.mean(fraud)}, std:{np.std(fraud)}')

In [ ]:
plot_distribution('card1', tr_non_fraud, tr_fraud, label_encoder=label_encoder['card1'])

In [ ]:
plot_distribution('card2', tr_non_fraud, tr_fraud, label_encoder=label_encoder['card2'])

In [ ]:
plot_distribution('addr1', tr_non_fraud, tr_fraud, label_encoder=label_encoder['addr1'])

In [ ]:
plot_distribution('TransactionID', tr_non_fraud, tr_fraud)

In [ ]:
# filt high TransactionAmt
filt_tr_non_fraud = tr_non_fraud[tr_non_fraud['TransactionAmt']<1500]
filt_tr_fraud = tr_fraud[tr_fraud['TransactionAmt']<1500]

plot_distribution('TransactionAmt', filt_tr_non_fraud, filt_tr_fraud)

In [ ]:
plot_distribution('P_emaildomain', tr_non_fraud, tr_fraud)

fraud_mail = tr_fraud['P_emaildomain']
unique, counts = np.unique(fraud_mail, return_counts=True)
print (dict(zip(unique, counts )))
print (label_encoder['P_emaildomain'].inverse_transform([16,19,36]))